In [1]:
import requests
import json
import duckdb
from config import get_api_token
import pandas as pd

In [2]:
# token, adjust tickets limits

app_token = get_api_token()
print(app_token)
base_url = 'https://data.lacity.org/resource/4f5p-udkv.json'

# get headers
headers = {
    'Accept': 'application/json',  
    'X-App-Token': app_token  
}
    
# Define the schema with default values
schema = {
    'agency': None,
    'agency_desc': None,
    'body_style': None,
    'body_style_desc': None,
    'color': None,
    'color_desc': None,
    'fine_amount': None,
    'issue_date': None,
    'issue_time': None,
    'loc_lat': None,
    'loc_long': None,
    'location': None,
    'make': None,
    'marked_time': None,
    'meter_id': None,
    'plate_expiry_date': None,
    'rp_state_plate': None,
    'ticket_number': None,
    'violation_code': None,
    'violation_description': None
}

def create_db_and_table():
    """Create DuckDB instance and tables based on schema including dimension tables."""
    con = duckdb.connect(database=':memory:')
    
    # Create fact table
    con.execute("""
        CREATE TABLE tickets (
            agency INTEGER,
            agency_desc VARCHAR,
            body_style VARCHAR,
            body_style_desc VARCHAR,
            color VARCHAR,
            color_desc VARCHAR,
            fine_amount DECIMAL,
            issue_date TIMESTAMP,
            issue_time VARCHAR,
            loc_lat FLOAT,
            loc_long FLOAT,
            location VARCHAR,
            make VARCHAR,
            marked_time VARCHAR,
            meter_id VARCHAR,
            plate_expiry_date VARCHAR,
            rp_state_plate VARCHAR,
            ticket_number VARCHAR,
            violation_code VARCHAR,
            violation_description VARCHAR
        )
    """)
    
    return con


def apply_schema(data, schema):
    """apply the schema to individual data entries."""
    return {key: data.get(key, schema[key]) for key in schema}

def insert_data(con, data):
    """insert data into DuckDB table."""
    placeholders = ', '.join(['?'] * len(data[0]))
    columns = ', '.join(data[0].keys())
    query = f'INSERT INTO tickets ({columns}) VALUES ({placeholders})'
    con.executemany(query, [tuple(item.values()) for item in data])

def fetch_pages(base_url, limit, pages, schema, headers, con):
    order = "issue_date ASC, issue_time ASC, ticket_number ASC"
    for page in range(pages):
        offset = limit * page
        url = f"{base_url}?$limit={limit}&$offset={offset}&$order={order}"
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            raw_data = response.json()
            normalized_data = [apply_schema(item, schema) for item in raw_data]
            insert_data(con, normalized_data)
            print(f"Fetched and inserted page {page + 1} with {len(normalized_data)} records.")
        else:
            print(f"Failed to fetch data on page {page + 1}: {response.status_code}")
            break



ZzVI2vN6lETzguOojTmivo03L


In [3]:
# Main execution
if __name__ == "__main__":
    con = create_db_and_table()
    fetch_pages(base_url, 100, 20, schema, headers, con)

Fetched and inserted page 1 with 100 records.
Fetched and inserted page 2 with 100 records.
Fetched and inserted page 3 with 100 records.
Fetched and inserted page 4 with 100 records.
Fetched and inserted page 5 with 100 records.
Fetched and inserted page 6 with 100 records.
Fetched and inserted page 7 with 100 records.
Fetched and inserted page 8 with 100 records.
Fetched and inserted page 9 with 100 records.
Fetched and inserted page 10 with 100 records.
Fetched and inserted page 11 with 100 records.
Fetched and inserted page 12 with 100 records.
Fetched and inserted page 13 with 100 records.
Fetched and inserted page 14 with 100 records.
Fetched and inserted page 15 with 100 records.
Fetched and inserted page 16 with 100 records.
Fetched and inserted page 17 with 100 records.
Fetched and inserted page 18 with 100 records.
Fetched and inserted page 19 with 100 records.
Fetched and inserted page 20 with 100 records.


In [4]:
# Execute a query to fetch the first 5 rows from the 'tickets' table
query = "SELECT * FROM tickets LIMIT 10"
# result = con.execute(query).fetchall()

# Execute a query and fetch the result as a Pandas DataFrame
df = con.execute(query).df()
display(df)

,agency,agency_desc,body_style,body_style_desc,color,color_desc,fine_amount,issue_date,issue_time,loc_lat,loc_long,location,make,marked_time,meter_id,plate_expiry_date,rp_state_plate,ticket_number,violation_code,violation_description
0,2,LAX CURRENT,PA,PASSENGER CAR,WH,None,NaN,1950-07-15,1550,33.943085,-118.399529,701 WORLD WAY,LAND,None,None,201910,CA,1127488375,None,None
1,1,WESTERN,SU,None,BK,BLACK,NaN,1962-01-30,1650,34.033726,-118.265366,8TH ST W/O HILL ST,JEEP,None,CB3235,202005,CA,1121858091,None,None
2,1,WESTERN,PA,PASSENGER CAR,BK,BLACK,NaN,1970-01-31,1840,34.036724,-118.268227,VENICE-PEMBROKE,HOND,None,None,202003,CA,1121875510,None,None
3,1,WESTERN,PA,PASSENGER CAR,GY,GREY,NaN,1972-01-22,1555,34.156452,-118.439163,14000 RIVERSIDE DR,TOYO,None,None,202009,CA,1121270581,None,None
4,1,WESTERN,PA,PASSENGER CAR,TA,None,NaN,1972-01-30,1805,34.045135,-118.256676,8TH ST W/O OLIVE,TOYO,None,None,202004,CA,1121858124,None,None
5,2,LAX CURRENT,TR,TRAILER,GY,GREY,0.0,2000-01-06,642,33.943085,-118.399529,700 WORLD WAY U/L,FORD,0000,None,202209,CA,1128903263,8939,None
6,54,54 - DOT - HOLLYWOOD,PA,PASSENGER CAR,BK,BLACK,0.0,2000-01-25,932,34.079315,-118.313660,5042 ROSEWOOD AV,SUBA,0000,None,202110,CA,1126402771,8069BS,None
7,1,WESTERN,PA,PASSENGER CAR,SI,None,0.0,2000-01-28,1125,34.185440,-118.610550,22216 VICTORY BL,HOND,0000,None,201905,CA,1125067753,4000A1,None
8,1,WESTERN,PA,PASSENGER CAR,GO,GOLD,0.0,2000-01-28,1700,34.188652,-118.588455,KITTRIDGE W/O DE SOT,ACUR,0000,None,202103,CA,1125033442,4000A1,None
9,1,WESTERN,PA,PASSENGER CAR,BK,BLACK,0.0,2000-02-05,1130,34.053646,-118.245514,32/BROADWAY,HOND,0000,None,202011,CA,1121626240,4000A1,None


In [5]:
# Close the connection
con.close()